# Automate HCPCS Formating for Prism

Every quarter, the business requests HCPCS and ICD10 Codes to be updated in the master list for **Prism**, **Que**, **Salesforce**, and **Essette**. This script is used to format a file for Essette.

This script will format the file given by the business into a format that is acceptable by Prism.

## Prerequisites
- No prior knowledge of Python is required to run this script. However, that knowledge will help you understand the logic and syntax of the script. Specifically, we will be using the **pandas** library to transpose the file given by the business to a DataFrame, then we will use that DataFrame to create a new DataFrame with the necessary information. We will also be using the **datetime** library specifically for naming conventions of our file.
- You will need to **rename** the file given by the business to **request.csv**. If this step is not done, then this script **will not run** properly.
- You will need to save this Jupyter Notebook and the file **request.csv** in the same directory, e.g. in the same location on your local machine.
- You will need a method to run Jupyter Notebooks on your local machine. This script assumes Anaconda has already been installed to your local machine. If Anaconda is not installed in your local machine, then you will need to install it via https://www.anaconda.com/docs/getting-started/anaconda/install.

# HCPCS Code - Data Cleanup

## Overview of the Process
This script will read the filename **request.csv** and load it into a DataFrame using the pandas library. 

It will then use that DataFrame to make a new DataFrame with the following schema:
- **StagedForRemoval** = 0 (Note: We are assuming codes are not Staged for Removal. If Removal is needed, then they will need to be changed manually)
- **TenantCode** = IP
- **TypeCode** = CD.Service
- **ParentCode** = HCPCS
- **Code** = the HCPCS Code listed in the file called **request.csv**
- **Description** = the Short Description listed in the file called **request.csv**
- **AlternateDescription** = the Long Description listed in the file called **request.csv**
- **StartDate** = the Code Start Date listed in the file called **request.csv**
- **EndDate** = the Code End Date listed in the file called **request.csv**

In [2]:
# Import Libraries
import pandas as pd
from datetime import datetime

today = datetime.today() # Get Today's Date
formatted_date = today.strftime("%m-%d-%Y") # Strip Today's Date in the MMDDYYYY Format

In [3]:
# Load the CSV file into a DataFrame
df = pd.read_csv("master-list-dec-2025.csv", dtype=str, keep_default_na = False)  # Replace with your actual file name

In [4]:
df.head()

,HCPCS,Product Type,HCPCS Product Category Parent,HCPCS Product Category Parent Code,HCPCS Product Category Child,HCPCS Product Category Child Code,Que Category,QUE Category Code,Long Description,Short Description,...,Labor Code,Action,Transition Categories,CMS Categories,Jurisdiction,Code Start Date,Code End Date,Code Run Out Date,Unnamed: 20,Unnamed: 21
0,A0021,NDME,Uncategorized,UCTG,Uncategorized,UCTG,Uncategorized,UCTG,"Ambulance service, outside state per mile, tra...",Outside state ambulance serv,...,0,CREATE,NDME,NA,PBM,10-Sep-96,,,,
1,A4206,DME,Medical Supplies,MDS,Miscellaneous Medical Supplies,MMS,Uncategorized,UCTG,"Syringe with needle, sterile, 1 cc or less, each",1 cc sterile syringe&needle,...,0,CREATE,RO,SU,D,1-Jan-08,,,,
2,A0080,NDME,Uncategorized,UCTG,Uncategorized,UCTG,Uncategorized,UCTG,"Non-emergency transportation, per mile - vehic...",Noninterest escort in non er,...,0,CREATE,NDME,NA,PBM,1-Jan-03,,,,
3,A4207,DME,Medical Supplies,MDS,Miscellaneous Medical Supplies,MMS,Uncategorized,UCTG,"Syringe with needle, sterile 2 cc, each",2 cc sterile syringe&needle,...,0,CREATE,RO,SU,D,1-Jan-07,,,,
4,A0090,NDME,Uncategorized,UCTG,Uncategorized,UCTG,Uncategorized,UCTG,"Non-emergency transportation, per mile - vehic...",Interest escort in non er,...,0,CREATE,NDME,NA,PBM,1-Jan-03,,,,


In [5]:
# Create a new DataFrame with the desired columns and transformations
hcpcs = pd.DataFrame({
    "HCPCS": df["HCPCS"],
    "Product Type": df["Product Type"],
    "HCPCS Product Category Parent": df["HCPCS Product Category Parent"],
    "HCPCS Product Category Parent Code": df["HCPCS Product Category Parent Code"],
    "HCPCS Product Category Child": df["HCPCS Product Category Child"],
    "HCPCS Product Category Child Code": df["HCPCS Product Category Child Code"],
    "Que Category": df["Que Category"],
    "QUE Category Code": df["QUE Category Code"],
    "Long Description": df["Long Description"],
	"Short Description": df["Short Description"],
    "Frequency": df["Frequency"],
    "Miscellaneous Code": df["Miscellaneous Code"],
    "Labor Code": df["Labor Code"],
    "Code Start Date": df["Code Start Date"], # Update to the correct Start Date from Master List # Added on October 21, 2025 
    # For Start Date, if Invalid or Empty, then the job will fail
    # Format for Date: YYYY-MM-DD or MM-DD-YY
    "Code End Date": df["Code End Date"], # Update to the correct Start Date from Master List # End Date can be blank # Added on October 21, 2025
    # Format for Date: YYYY-MM-DD or MM-DD-YY
    # End Date, if Invalid, then the job will fail
    "Action": df["Action"]										
})



# If you wish to preview the result, uncomment the next line by removing the preceeding #
#print(hcpcs.head())

In [6]:
# Clean the Upload File
# Remove Special Characters
hcpcs["HCPCS"] = hcpcs["HCPCS"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)
hcpcs["Long Description"] = hcpcs["Long Description"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)
hcpcs["Short Description"] = hcpcs["Short Description"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)

# Date Format
# Check the Format of the Master List Date
df["Code Start Date"]
# Convert Datetime into Desired Format
hcpcs["Code Start Date"] = pd.to_datetime(hcpcs["Code Start Date"], format = "%d-%b-%y", errors = "coerce")
hcpcs["Code Start Date"] = hcpcs["Code Start Date"].dt.strftime("%Y-%m-%d")

hcpcs["Code End Date"] = pd.to_datetime(hcpcs["Code End Date"], format = "%d-%b-%y", errors = "coerce")
hcpcs["Code End Date"] = hcpcs["Code End Date"].dt.strftime("%Y-%m-%d")

In [7]:
# Query for specific HCPCS
filtered = hcpcs[hcpcs["HCPCS"].isin(["A4288"])]
filtered

,HCPCS,Product Type,HCPCS Product Category Parent,HCPCS Product Category Parent Code,HCPCS Product Category Child,HCPCS Product Category Child Code,Que Category,QUE Category Code,Long Description,Short Description,Frequency,Miscellaneous Code,Labor Code,Code Start Date,Code End Date,Action
95,A4288,DME,Medical Equipment and Supplies,MES,Breast Pumps,BRP,Breast Pumps A4280 - A4287,BPSA,"Valve for breast pump, replacement",Replacement breastpump valve,,0,0,2025-10-01,NaN,Create


In [8]:
# Create the filename
filename = "prism_hcpc_codes_" + formatted_date

In [9]:
# Load the Cleaned Data into a New CSV File
#hcpcs.to_csv(filename + "with_commas.csv", index=False)

hcpcs.to_csv(filename + ".csv", index=False)


# HCPCS Transition Category - Data Cleanup

## Overview of the Process
This script will read the filename **request.csv** and load it into a DataFrame using the pandas library. 

It will then use that DataFrame to make a new DataFrame with the following schema:
- **HCPCS** = the HCPCS Code listed in the file called **request.csv**
- **Mod1** = Blank Field
- **HCPCS Description** = the Long Description listed in the file called **request.csv**
- **HCPCS Category** = the HCPCS Product Category Parent in the file called **request.csv**
- **Transition Codes** = the Transition Category in the file called **request.csv**
- **Monthly Frequency** = the Frequency in the file called **request.csv**



In [11]:
# Reset the new_df with the desired columns
icd10 = pd.DataFrame({
    "HCPCS": df["HCPCS"],
    "Mod1": None,
    "HCPCS Description": df["Long Description"],
    "HCPCS Category": df["HCPCS Product Category Parent"],
    "Transition Codes": df["Transition Categories"],
    "Monthly Frequency": df["Frequency"]
})

icd10["Monthly Frequency"] = icd10["Monthly Frequency"].apply(
    lambda x: "Y" if str(x).strip().upper() == "MONTHLY" else "N"
)

# Clean the Data
icd10["HCPCS"] = icd10["HCPCS"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)
icd10["HCPCS Description"] = icd10["HCPCS Description"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)

# Create the filename
filename = "prism_hcpc_transition_category_" + formatted_date

# Load the Cleaned Data into a New CSV File
#icd10.to_csv(filename + "with_commas.csv", index=False)

#icd10.to_csv(filename + ".csv", index=False, sep='|')
icd10.to_csv(filename + ".csv", index=False)